# National Park Service MCP Server Integration workbook

In [83]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [84]:
nps_api_server_params = [
    {"command": "uv", "args": ["run","nps.py"]}
]

# server1 = [
#     {"command": "uvx", "args": ["abc.py"]},
#     {"command": "uvx", "args": ["xyz.py"]}
# ]

In [85]:
nps_api_server = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in nps_api_server_params]

In [86]:
mcp_servers = nps_api_server
#mcp_servers = server1 + server2 +server3....

In [87]:
async def nps_agent(mcp_servers) -> Agent:
    instructions = """You are a National Park Service (NPS) Assistant, 
    an expert guide specializing in America's national parks, monuments, historic sites, and recreational areas. 
    Your mission is to help visitors discover, plan, and explore the incredible natural 
    and cultural treasures protected by the National Park Service.
    You use available tools to call NPS website API to provide accurate information
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [88]:


async def nps_tool(mcp_servers) -> Tool:
    researcher = await nps_agent(mcp_servers)
    return researcher.as_tool(
            tool_name="NPS-Researcher",
            tool_description="This tool researches online for park information, \
                either based on your specific request to look into a certain park, \
                Describe the park name to get further information"
        )



In [90]:
message = "Where can i get passport stamps in North cascade national parks"

for server in mcp_servers:
    print (server)
    await server.connect()
researcher = await nps_agent(mcp_servers) 
with trace("NPS-Researcher"):
    result = await Runner.run(researcher, message, max_turns=30)
display(Markdown(result.final_output))

You can get passport stamps at the following locations in North Cascades National Park:

1. Forest Info Center with HQ Administration Building
2. Golden West Visitor Center
3. North Cascades Visitor Center
4. Wilderness Information Center

These visitor centers will have the National Park passport stamps available for you. If you need more information about the park, you can visit the official website here: https://www.nps.gov/noca/index.htm